# Building a Neural Network 'By Hand' in Numpy to Classify Irises

The only imports we need!

In [1]:
import numpy as np
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

The data is the classic iris dataset- this csv was downloaded from [kaggle.](https://www.kaggle.com/datasets/saurabh00007/iriscsv/code)

In [2]:
data = read_csv('Iris.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('Species', axis=1), data['Species'], test_size=0.2, random_state=9)

## Defining a Node Class and RelU and One-to-One (none) Activation Functions

In [3]:
def relu(x):
    if x <= 0: return 0
    else: return x

In [4]:
def onetoone(x):
    return x

In [5]:
class Node():
    def __init__(self, weights, bias, activationFn):
        self.bias = bias
        self.activationFn = activationFn
        self.weights = weights
    def calculate(self, inputs):
        return float(self.activationFn(np.dot(inputs, self.weights) + self.bias))
    

## Defining a SoftMax Function

In [6]:
def softmax(outputs):
    return outputs/outputs.sum()

## One-Hot Encoding the Targets

In [7]:

encoder = OneHotEncoder(sparse=False)
y_train_transformed = encoder.fit_transform(np.array(y_train).reshape(-1,1))

C:\Users\Mark\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Scaling the Inputs

In [8]:
X_train['SepalLengthCm'] = X_train['SepalLengthCm'] / X_train['SepalLengthCm'].max() - 0.5
X_train['SepalWidthCm'] = X_train['SepalWidthCm'] / X_train['SepalWidthCm'].max() - 0.5
X_train['PetalLengthCm'] = X_train['PetalLengthCm'] / X_train['PetalLengthCm'].max() - 0.5
X_train['PetalWidthCm'] = X_train['PetalWidthCm'] / X_train['PetalWidthCm'].max() - 0.5

## Defining a Neural Network Class

In [9]:
class NeuralNetwork():
    def __init__(self, layers):
        '''Takes a list of lists of nodes and creates a Neural Network object'''
        self.layers = layers
    def classify(self, inputs):
        '''Takes a numpy array of inputs and returns predictions'''
        for layer in self.layers:
            arr = np.zeros(len(layer))
            i = 0
            for node in layer:
                arr[i] = node.calculate(inputs)
                i += 1
            inputs = arr
        return softmax(inputs)
    def train(self, learning_rate, X_train, targets):
        ls = []
        
        for i in X_train.index:
            inputs = np.array([X_train.loc[i].SepalLengthCm,
                               X_train.loc[i].SepalWidthCm,
                               X_train.loc[i].PetalLengthCm,
                               X_train.loc[i].PetalWidthCm])
            arr = self.classify(inputs=inputs)
            ls.append(arr)
        preds = np.array(ls)
        #Hardcoded 3 Outputs
        X_train['obs0'], X_train['obs1'], X_train['obs2'] = targets[:,0], targets[:,1], targets[:,2]
        X_train['preds0'], X_train['preds1'], X_train['preds2'] = preds[:,0], preds[:,1], preds[:,2]
        for n in range(len(self.layers[-1])):
            d_crossentropy_wrt_bias = 0
            d_crossentropy_wrt_weights = np.zeros(len(self.layers[-2])) 
            d_crossentropy_wrt_node_biases = np.zeros(len(self.layers[-2]))
            d_crossentropy_nodeweights_total = np.zeros((len(self.layers[-2]), len(inputs)))
            
            
            for i in X_train.index:
                d_crossentropy_wrt_bias += -X_train.loc[i][str('obs' + str(n))] * (1 - X_train.loc[i][str('preds' + str(n))])
                node_values = np.zeros(0)
                
                d_crossentropy_wrt_node_weights = np.zeros((len(self.layers[-2]), len(inputs)))
                
                for j in range(len(self.layers[-2])):
                    node_value = self.layers[-2][j].calculate(inputs)
                    node_bias = self.layers[-2][j].bias
                    node_weights = self.layers[-2][j].weights
                    node_values = np.append(node_values, node_value)
                    
                    if node_value > 0: #Derivative of Relu is 1 (And divides out) if y > 0, otherwise derivative is 0 and cancels out, and no need to update total gradient.
                        d = self.layers[-1][n].weights * (X_train.loc[i][str('obs' + str(n))] / X_train.loc[i][str('preds' + str(n))])
                        d_crossentropy_wrt_node_biases += d
                        d_crossentropy_wrt_node_weights[j] = d[j] * inputs
                
                    
                        
                    
                d_crossentropy_wrt_weights += -X_train.loc[i][str('obs' + str(n))] *(1 - X_train.loc[i][str('preds' + str(n))]) * node_values
                d_crossentropy_nodeweights_total += d_crossentropy_wrt_node_weights
                
            bias_step = d_crossentropy_wrt_bias * learning_rate
            weights_steps = d_crossentropy_wrt_weights * learning_rate
            node_biases_step = d_crossentropy_wrt_node_biases * learning_rate
            node_weights_step = d_crossentropy_nodeweights_total * learning_rate
            
            self.layers[-1][n].bias += bias_step
            self.layers[-1][n].weights += weights_steps
            
            for nd in range(len(self.layers[-2])):
                self.layers[-2][nd].bias += node_biases_step[nd]
                self.layers[-2][nd].weights += node_weights_step[nd]
        

## Initializing the Hidden Layer

In [10]:
hidden_node_1 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )
hidden_node_2 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )
hidden_node_3 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )
hidden_node_4 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )
hidden_node_5 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )
hidden_node_6 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )
hidden_node_7 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )
hidden_node_8 = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=relu
                     )


hidden_layer = [hidden_node_1, hidden_node_2, hidden_node_3, hidden_node_4, hidden_node_5, hidden_node_6, hidden_node_7, hidden_node_8]

## Initializing the Output Layer

In [11]:
setosa_node = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=onetoone
                     )
virginica_node = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5),
                                        5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=onetoone
                     )
versicolor_node = Node(weights=np.array([5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5),
                                         5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5),
                                       5 * (np.random.rand() - 0.5), 
                                       5 * (np.random.rand() - 0.5)]),
                     bias=0,
                     activationFn=onetoone
                     )
output_layer = [setosa_node, versicolor_node, virginica_node]

In [12]:
neural_net = NeuralNetwork(layers=[hidden_layer,output_layer])

In [52]:
for i in range(200): neural_net.train(X_train=X_train, targets=y_train_transformed, learning_rate=0.01)

In [49]:
virginica = np.array([6.3,3.3,6.0,2.5])
versi = np.array([5.0,2.3,3.3,1.0])
setosa = np.array([4.8,3.0,1.4,0.1])

In [53]:
print(neural_net.classify(inputs=virginica))
print(neural_net.classify(inputs=versi))
print(neural_net.classify(inputs=setosa))

[0.30558037 0.35733001 0.33708962]
[0.31283835 0.34578444 0.34137721]
[0.31695327 0.34059873 0.342448  ]


In [51]:
preds_setosa = []
preds_virginica = []
preds_versicolor = []
for i in X_test.index:

    inputs = np.array([X_test.loc[i].SepalLengthCm,
                       X_test.loc[i].SepalWidthCm,
                       X_test.loc[i].PetalLengthCm,
                       X_test.loc[i].PetalWidthCm])
    preds = neural_net.classify(inputs=inputs)
    preds_setosa.append(preds[0])
    preds_versicolor.append(preds[1])
    preds_virginica.append(preds[2])

X_test['Species'] = y_test
X_test['setosa'] = preds_setosa
X_test['versicolor'] = preds_versicolor
X_test['virginica'] = preds_virginica
X_test

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,setosa,versicolor,virginica
135,136,7.7,3.0,6.1,2.3,Iris-virginica,0.303692,0.360500,0.335808
90,91,5.5,2.6,4.4,1.2,Iris-versicolor,0.310110,0.351409,0.338481
145,146,6.7,3.0,5.2,2.3,Iris-virginica,0.304453,0.358427,0.337120
147,148,6.5,3.0,5.2,2.0,Iris-virginica,0.305820,0.357203,0.336977
60,61,5.0,2.0,3.5,1.0,Iris-versicolor,0.311966,0.347138,0.340896
37,38,4.9,3.1,1.5,0.1,Iris-setosa,0.316896,0.341468,0.341636
26,27,5.0,3.4,1.6,0.4,Iris-setosa,0.315416,0.343490,0.341094
3,4,4.6,3.1,1.5,0.2,Iris-setosa,0.316559,0.341764,0.341676
75,76,6.6,3.0,4.4,1.4,Iris-versicolor,0.308885,0.353170,0.337945
9,10,4.9,3.1,1.5,0.1,Iris-setosa,0.316896,0.341468,0.341636
